## 创建chroma数据库

In [1]:
import chromadb
import os
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb import Settings

db_path = "./news_db"  # 离线数据库存放路径
news_db = "news"  # 数据库名称，用于后续调用
if not os.path.exists(db_path):  # 如果路径不存在则创建该路径
    os.mkdir(db_path)
settings = Settings(allow_reset=True)  # 数据库设置，允许使用reset指令
client = chromadb.PersistentClient(path=db_path, settings=settings)  # 根据指定路径和设置创建chroma数据库
news_collection = client.get_or_create_collection(name=news_db, metadata={"hnsw:space": "cosine"})  # 创建collection，并指定距离计算公式为cosine

## 指定embedding模型

In [2]:
from sentence_transformers import SentenceTransformer
import torch

# 查询是否可使用GPU，否则使用CPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('embedding model using cuda')
else:
    device = torch.device('cpu')
    print('embedding model using cpu')

# 设定embedding model
embedding_model = SentenceTransformer('embedding_model/bge-small-zh-v1_5', device)
query_instruction = "为这个句子生成表示以用于检索相关文章："

embedding model using cpu


## 读取数据并将数据写入数据库

In [3]:
# 读取新闻数据
import json
json_file = 'dataset/news.json'  # 原数据路径
with open(json_file) as f:
    data = json.load(f)['event_summary']  # 读取新闻数据，data是一个list，每个元素是个dict

# 新闻数据写入数据库
from tqdm import tqdm
batch_num = 2  # 可根据电脑性能调整batch_num大小，batch_num决定了同时将文本转化为embeddings的文本数量
for i in tqdm(range(0, len(data), batch_num)):  # 根据batch大小，循环转化文本向量并写入数据库
    summary_batch = []
    ids_batch = []
    metadata_batch = []
    for j in range(min(len(data) - i, batch_num)):
        summary_batch.append(data[i+j].pop('summary'))  # summary会被放置于documents
        ids_batch.append(data[i+j].pop('ID'))  # ID需单独存放
        metadata_batch.append(data[i+j])  # 剩下的数据部分入metadata
    embeddings = embedding_model.encode(summary_batch, normalize_embeddings=True)  # 使用embedding模型将指定文本转化为向量
    # 将一个batch的数据写入数据库
    news_collection.upsert(
    embeddings=embeddings,
    documents=summary_batch,
    metadatas=metadata_batch,
    ids=ids_batch
)

100%|██████████| 1000/1000 [00:23<00:00, 42.92it/s]


### 检查写入

In [4]:
print("数据库内数据的条数:", news_collection.count())

数据库内数据的条数: 2000


## 请求召回测试

In [5]:
queries = ["APP的过渡期是多久", "美国哪个银行倒闭了"]
results_num = 3  # 返回相关结果条数

q_embeddings = embedding_model.encode([query_instruction+q for q in queries], normalize_embeddings=True)  # 转化请求数据为向量

results = news_collection.query(
    query_embeddings=q_embeddings,
    n_results=results_num,
)

print(results)  # 打印返回结果

{'ids': [['64fa9b2bb82641eb8ecbebe2', '64fa9b32b82641eb8ecc0a35', '64fa9b2cb82641eb8ecbeee1'], ['64fa9b2fb82641eb8ecc0227', '64fa9b2bb82641eb8ecbec4e', '64fa9b32b82641eb8ecc0870']], 'distances': [[0.3519214391708374, 0.4704177975654602, 0.512534499168396], [0.32318347692489624, 0.4799996614456177, 0.497724711894989]], 'metadatas': [[{'event': '昨晚，工信部发布通知，从2023年9月起，APP需按照国家法律法规要求备案，2024年4月至2024年6月底，电信主管部门将对APP备案情况开展监督检查。', 'text': '记者：实习记者 夏骅，正文：昨晚，工信部发布组织开展移动互联网应用程序备案工作的通知，从9月起组织开展APP备案工作。在10个月的备案过渡期后，网络接入服务提供者、应用分发平台、智能终端生产企业不得为未履行备案手续的APP提供网络接入、分发、预置等服务。自2000年起，依据《互联网信息服务管理办法》（国务院令第292号）规定，电信主管部门对从事互联网信息服务的网站开展备案核准工作。随着移动互联网快速发展，APP已成为互联网信息服务的重要载体。工信部表示，APP应按照国家法律法规要求，向电信主管部门参照网站备案的方式履行备案手续。据介绍，APP主办者在填写有关备案材料并实名核验后，由其网络接入服务提供者或应用分发平台通过“国家互联网基础资源管理系统”，向APP主办者住所所在地通信管理局在线提交备案申请，APP主办者无需到通信管理局窗口排队办理。APP需要何时完成备案？工信部表示，综合考虑APP主办者、网络接入服务提供者、应用分发平台、智能终端生产企业实际业务情况，预留10个月时间作为APP备案工作的过渡期。2023年9月至2024年3月底，在通知发布前开展业务的APP向其住所所在地省级通信管理局履行备案手续。2024年4月至2024年6月底，电信主管部门将组织对APP备案情况开展监督检查，对仍未履行备案手续的APP